In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

In [ ]:
urls = [
    {'url':'https://shipandbunker.com/prices/apac/sea/sg-sin-singapore#VLSFO', 'symbol':'MBI0798', 'boerse':'mbi', 'fuel_type':'VLSFO'},
    {'url':'https://shipandbunker.com/prices/apac/sea/sg-sin-singapore#_IFO380', 'symbol':'MBI0796', 'boerse':'mbi', 'fuel_type':'IFO380'},
    {'url':'https://shipandbunker.com/prices/emea/nwe/nl-rtm-rotterdam#VLSFO', 'symbol':'MBI0797', 'boerse':'mbi', 'fuel_type':'VLSFO'},
    {'url':'https://shipandbunker.com/prices/emea/nwe/nl-rtm-rotterdam#IFO380', 'symbol':'MBI0795', 'boerse':'mbi', 'fuel_type':'IFO380'}
]

In [ ]:
all_data = []

for item in urls:
    url = item['url']
    symbol = item['symbol']
    boerse = item['boerse']
    fuel_type = item['fuel_type']
        
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        data = []
        headers = ["symbol", "boerse", "datum", "letzter"]
        
        #price
        price_tag = soup.find('td', headers = f'price-{fuel_type}')
        price = price_tag.text.strip() if price_tag else "price not found"

        #date
        date_tag = soup.find('th', class_='date')
        
        raw_date = date_tag.text.strip() if date_tag else "date not found"
        raw_date = raw_date[2:]
        
        current_year = datetime.now().year
        
        formatted_date = f"{current_year}-{datetime.strptime(raw_date, '%b %d').strftime('%m-%d')}"
        
        data.append([symbol, boerse, formatted_date, price])
        
        #check up
        #print(symbol, boerse, date, price_tag)

    all_data.extend(data)

df = pd.DataFrame(all_data, columns=headers)
df.to_excel("shipandbunker.xlsx", index=False)

print("Excel-Datei erfolgreich erstellt in shipandbunker.xlsx")